In [1]:
#-*-coding:utf-8 -*-
#所有需要的库文件添加该处
import pandas as pd
from six import StringIO
import matplotlib.pyplot as plt
import  numpy as np
import os
from datetime import datetime,timedelta
from pyecharts import Line,Kline,Bar,Overlap,Grid
from jqdatasdk import *
import requests
import tushare as ts
pd.set_option('precision',3)

#============一下为指数估值分析
#所用文件目录如下：
stock_basic_info_dir = "C:\\quanttime\\data\\basic_info\\all_stock_info.csv"
index_basic_info_dir = "C:\\quanttime\\data\\basic_info\\index_all_info_for_analyse.csv"
index_valuation_dir = "C:\\quanttime\\data\\index\\index_valuation\\"
stock_valuation_dir = "C:\\quanttime\\data\\finance\\valuation\\"
index_k_dir = "C:\\quanttime\\data\\index\\jq\\"

#需要分析的指数添加到该list中
analyse_index_list = ["000001.XSHG", "000300.XSHG", "000905.XSHG", "399001.XSHE",\
                      "399005.XSHE", "399006.XSHE", "399016.XSHE","399300.XSHE",\
                      "399975.XSHE","399368.XSHE","399959.XSHE","399967.XSHE","399241.XSHE",\
                      "000015.XSHG","000016.XSHG","000813.XSHG","000814.XSHG","000815.XSHG",\
                     "000005.XSHG","000006.XSHG","000007.XSHG","000015.XSHG","000816.XSHG",\
                     "000948.XSHG","000043.XSHG"]

#加载index 基本信息
df_index_info = pd.read_csv(index_basic_info_dir,index_col=["code"],encoding="gbk")
#结果按照如下column排列：
#code，名称，Pb，5%分位，PB_min，PB_max，PB_中值，PB_方差，Pe，5%分位，PE_min，PE_max，PE_均值，PE_方差
general_result = []
for index in analyse_index_list:
    index_valuation = index_valuation_dir + index + '.csv'
    #df_index_valuation = pd.read_csv(index_valuation,index_col=['date'])
    #df_index_valuation = df_index_valuation[~df_index_valuation.reset_index().duplicated().values]
    df_index_valuation = pd.read_csv(index_valuation)
    df_index_valuation = df_index_valuation.drop_duplicates("date")
    df_index_valuation = df_index_valuation.set_index("date")
    
    name = df_index_info.loc[index,['display_name']].display_name
    pb = df_index_valuation.loc[df_index_valuation.index[len(df_index_valuation.index)-1],['PB']].PB
    pb_quantile_5 = df_index_valuation.PB.quantile(0.05)
    pb_min = '%r(%r)'%(df_index_valuation.PB.min(),df_index_valuation.PB.idxmin())
    pb_max = '%r(%r)'%(df_index_valuation.PB.max(),df_index_valuation.PB.idxmax())
    pb_median = df_index_valuation.PB.median()
    pb_std = df_index_valuation.PB.std()
    pe = df_index_valuation.loc[df_index_valuation.index[len(df_index_valuation.index)-1],['PE']].PE
    pe_quantile_5 = df_index_valuation.PE.quantile(0.05)
    pe_min = '%r(%r)'%(df_index_valuation.PE.min(),df_index_valuation.PE.idxmin())
    pe_max = '%r(%r)'%(df_index_valuation.PE.max(),df_index_valuation.PE.idxmax())
    pe_median = df_index_valuation.PE.median()
    pe_std = df_index_valuation.PE.std()
    general_result.append([name,pb,pb_quantile_5,pb_min,pb_max,pb_median,pb_std,pe,pe_quantile_5,pe_min,pe_max,pe_median,pe_std])
df_columns=[u'名称', "PB", u'5%分位', u'PB最小', u'PB最大', u'PB中位数', u'PB标准差', \
                     "PE", u'5%分位', u'PE最小', u'PE最大', u'PE中位数', u'PE标准差']
pe_df = pd.DataFrame(data=general_result,index=analyse_index_list,columns=df_columns)
pe_df.index.name = "code"
pe_df

,名称,PB,5%分位,PB最小,PB最大,PB中位数,PB标准差,PE,5%分位,PE最小,PE最大,PE中位数,PE标准差
code,,,,,,,,,,,,,
000001.XSHG,上证指数,1.22,1.323,1.21('2014/5/7'),7.14('2007/10/15'),1.91,1.098,11.80,10.420,9.04('2014/4/28'),72.54('2007/10/15'),17.560,11.468
000300.XSHG,沪深300,1.24,1.300,1.17('2014-05-07'),7.46('2007-10-16'),1.78,1.214,10.92,9.312,8.18('2014-05-19'),65.63('2007-10-31'),14.235,10.333
000905.XSHG,中证500,1.52,1.860,1.44('2008-11-04'),5.89('2015-06-15'),2.79,0.895,17.54,22.107,17.54('2019-01-03'),103.75('2008-01-15'),38.690,19.228
399001.XSHE,深证成指,1.96,1.850,1.67('2005-11-15'),8.82('2007-10-16'),2.82,1.262,17.40,12.413,10.61('2005-07-11'),63.44('2007-10-16'),21.920,11.136
399005.XSHE,中小板指,2.58,3.062,2.39('2008-11-04'),8.17('2007-10-10'),4.20,1.157,20.45,23.440,19.44('2008-10-27'),80.41('2008-01-15'),34.910,11.791
399006.XSHE,创业板指,3.43,3.050,2.51('2012-12-03'),15.01('2015-06-03'),4.80,1.946,31.10,33.220,27.99('2012-12-03'),139.96('2015-06-03'),55.660,19.446
399016.XSHE,深证创新指数,2.57,2.790,2.57('2019-01-03'),4.79('2017-11-13'),4.16,0.551,18.33,19.826,18.33('2019-01-03'),40.29('2017-11-13'),33.285,5.988
399300.XSHE,沪深300,1.24,1.300,1.17('2014-05-07'),7.46('2007-10-16'),1.79,1.214,10.92,9.312,8.18('2014-05-19'),65.63('2007-10-31'),14.260,10.332
399975.XSHE,中证全指证券公司指数(四级行业),1.32,1.338,1.1('2018/10/18'),6.11('2015/6/8'),2.44,0.988,40.97,15.477,12.4('2015/9/15'),82.1('2013/9/12'),37.290,14.495
